# TME HMM : décodage des lettres

Le but est le même que dans le TME précédent: apprendre à classer les lettres les lettres manuscrites enregistrées par un stylo intelligent.

Les étapes sont un peu les mêmes que dans le TME précédent... Avec un modèle plus complexe.

In [78]:
import numpy as np
import pickle as pkl
import matplotlib.pyplot as plt

with open('ressources/lettres.pkl', 'rb') as f:
    data = pkl.load(f, encoding='latin1') 
X = np.array(data.get('letters')) # récupération des données sur les lettres
Y = np.array(data.get('labels')) # récupération des étiquettes associées 

nCl = 26

In [79]:
# affichage d'une lettre (cf TME précédent)
def tracerLettre(let):
    a = -let*np.pi/180; # conversion en rad
    coord = np.array([[0, 0]]); # point initial
    for i in range(len(a)):
        x = np.array([[1, 0]]);
        rot = np.array([[np.cos(a[i]), -np.sin(a[i])],[ np.sin(a[i]),np.cos(a[i])]])
        xr = x.dot(rot) # application de la rotation
        coord = np.vstack((coord,xr+coord[-1,:]))
    plt.figure()
    plt.plot(coord[:,0],coord[:,1])
    #plt.savefig("exlettre.png")
    return

In [80]:
# discrétisation (cf TME précédent)
def discretise(x, d):
    # votre code :
    intervalle = 360 / d
    xd = [np.floor(x[i] / intervalle) for i in range(len(x))]
    return np.array(xd)

## A. Apprentissage d'un modèle connaissant les états cachés du système

### A1. Hypothèse gauche-droite

On fait l'hypothèse que les états sont connus... Alors que ce n'est pas le cas. Mais il existe des stratégies simples (et parfois efficaces) pour attribuer arbitrairement des états sur les chaines.
La plus classique est l'hypothèse gauche-droite, qui est bien adaptée aux signaux courts et non périodiques:

* On définit le nombre d'états N
* On découpe chaque série d'observations en N portions à peu près égales
* On affecte l'état 0 au début, puis on incrémente jusqu'à l'état N pour la dernière portion de la chaine

Sur un exemple:

```
X0 = [ 1.  9.  8.  8.  8.  8.  8.  9.  3.  4.  5.  6.  6.  6.  7.  7.  8.  9.  0.  0.  0.  1.  1.]
S0 = [ 0.  0.  0.  0.  0.  0.  1.  1.  1.  1.  1.  1.  2.  2.  2.  2.  2.  3.  3.  3.  3.  3.  3.]
```

Au niveau de la mise en oeuvre, vous définirez la méthode ```def initGD(X,N):```qui prend un ensemble de séquences d'observations et qui retourne l'ensemble des séquences d'états. Pour chaque séquence $x$ vous pouvez utiliser:
```
np.floor(np.linspace(0,N-.00000001,len(x)))
```

In [81]:
def initGD(x,N):
    return np.floor(np.linspace(0,N-.00000001,len(x)))
    
# 14 observations aléatoires
x = np.floor(np.random.rand(14)*10)
print(initGD(x,4))

[0. 0. 0. 0. 1. 1. 1. 2. 2. 2. 3. 3. 3. 3.]


In [89]:
# Construire la base Q des états, initialisés en Gauche-Droite correspondant aux observations X
N = 5 # nombre d'états

Q = # A compléter
print(X[0],"\n",Q[0])

[ 36.214493 347.719116 322.088898 312.230957 314.851013 315.487213
 313.556702 326.534973 141.288971 167.606689 199.321594 217.911087
 226.443298 235.002472 252.354492 270.045654 291.665161 350.934723
  17.892815  20.281025  28.207161  43.883423  53.459026] [0. 0. 0. 0. 0. 1. 1. 1. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 4. 4. 4. 4. 4.]


Check:
```
[ 36.214493 347.719116 322.088898 312.230957 314.851013 315.487213
 313.556702 326.534973 141.288971 167.606689 199.321594 217.911087
 226.443298 235.002472 252.354492 270.045654 291.665161 350.934723
  17.892815  20.281025  28.207161  43.883423  53.459026] 
[0. 0. 0. 0. 0. 1. 1. 1. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 4. 4. 4. 4. 4.]
```

### A2. Apprentissage

Etant donné la structure d'un MMC:
* les observations n'influencent pas les états: les matrices $\Pi, A$ s'obtiennent comme dans un modèle de Markov simple 
* chaque observation ne dépend que de l'état courant

La nature des données nous pousse à considérer des lois de probabilités discrètes quelconques pour les émissions. L'idée est donc de procéder par comptage en définissant la matrice $B$ comme suit:
* $K$ colonnes (nombre d'observations), $N$ lignes (nombre d'états)
* Chaque ligne correspond à une loi d'émission pour un état (ie, chaque ligne somme à 1)
Ce qui donne l'algorithme:
1. $b_{ij}$ = comptage des émissions depuis l'état $s_i$ vers l'observation $x_j$
1. normalisation des lignes de $B$

Donner le code de la fonction ```def learnHMM(allX, allS, N, K):``` qui apprend un modèle à partir d'un ensemble de couples (seq. d'observations, seq. d'états)

***Variante stabilisée***: afin d'éviter les transitions à probabilité nulle et de régulariser l'ensemble du système, il est intéressant d'initialiser les matrices de transition à une valeur non nulle. Cette initialisation *spéciale* peut être faite de manière optionnelle en utilisant les arguments par défaut de python:



In [99]:
def learnHMM(allx, allq, N, K, initTo0=False, eps = 1e-5):
    if initTo0:
        A = np.zeros((N,N))
        B = np.zeros((N,K))
        Pi = np.zeros(N)
    else:
        A = np.zeros((N,N)) # np.ones((N,N))*eps => non pertinent pour un modèle GD
        B = np.ones((N,K))*eps
        Pi = np.ones(N)*eps
    # suite de votre code
    pass

    return Pi , A, B


In [100]:
# le nombre d'états a été fixé au dessus (à 5)
K = 10 # discrétisation (=10 observations possibles)
Xd = discretise(X,K)
Pi, A, B = learnHMM(Xd[Y=='a'],Q[Y=='a'],N,K, True)

print(Pi,"\n", np.around(A,decimals=2), "\n", np.around(B,decimals=2))

[1. 0. 0. 0. 0.] 
 [[0.79 0.21 0.   0.   0.  ]
 [0.   0.76 0.24 0.   0.  ]
 [0.   0.   0.77 0.23 0.  ]
 [0.   0.   0.   0.76 0.24]
 [0.   0.   0.   0.   1.  ]] 
 [[0.06 0.02 0.   0.   0.   0.   0.   0.04 0.49 0.4 ]
 [0.   0.04 0.   0.13 0.09 0.13 0.02 0.09 0.41 0.09]
 [0.   0.   0.   0.02 0.12 0.5  0.31 0.04 0.   0.  ]
 [0.07 0.   0.   0.   0.   0.   0.26 0.33 0.2  0.15]
 [0.73 0.12 0.   0.   0.   0.   0.   0.02 0.02 0.12]]


Validation sur les séquences de la classe 'a':
```
  Pi=[ 1.  0.  0.  0.  0.]

  A=[[0.79 0.21 0.   0.   0.  ]
     [0.   0.76 0.24 0.   0.  ]
     [0.   0.   0.77 0.23 0.  ]
     [0.   0.   0.   0.76 0.24]
     [0.   0.   0.   0.   1.  ]] 

  B=[[0.06 0.02 0.   0.   0.   0.   0.   0.04 0.49 0.4 ]
     [0.   0.04 0.   0.13 0.09 0.13 0.02 0.09 0.41 0.09]
     [0.   0.   0.   0.02 0.12 0.5  0.31 0.04 0.   0.  ]
     [0.07 0.   0.   0.   0.   0.   0.26 0.33 0.2  0.15]
     [0.73 0.12 0.   0.   0.   0.   0.   0.02 0.02 0.12]]
```

## B. Algorithme de décodage, Viterbi

Retrouver **la sequence d'état caché la plus probable** ayant permis de générer la séquence d'observation.

Rappels sur l'algorithme Viterbi (1967):

- Il sert à estimer la séquence d'états la plus probable étant donnés les observations et le modèle.
- Il peut servir à approximer la probabilité de la séquence d'observation étant donné le modèle. 

1\. Initialisation (avec les indices à 0 en python): 

$$\begin{array}{ccccccccc} \delta_{0} (i) &=& \log \pi_{i} +\log b_{i} (x_{1}) \\ \Psi_{0}(i) &=& -1 \mbox{ Note: -1 car non utilisé normalement} \end{array}$$

2\. Récursion: 

$$ \begin{array}{ccccccccc} \delta_{t} (j) &=&\displaystyle \left[\max_{i} \delta_{t-1}(i) + \log a_{ij}\right] + \log b_{j}(x_{t}) \\ \Psi_{t}(j) &=&\displaystyle \arg\max_{i\in [1,\ N]} \delta_{t-1} (i) + \log a_{ij} \end{array}$$

3\. Terminaison (indices à {$T-1$} en python) 

$$ S^{\star} = \max_{i} \delta_{T-1}(i)$$

4\. Chemin $$\begin{array}{ccccccccc} s_{T-1}^{\star} & = &\displaystyle \arg\max_{i} \delta_{T-1}(i) \\ s_{t}^{\star} & = & \displaystyle \Psi_{t+1}(s_{t+1}^{\star}) \end{array}$$

L'estimation de $\log p(x_0^{T-1} | \lambda)$ est obtenue en cherchant la plus grande probabilité dans la dernière colonne de $\delta$. Donner le code de la méthode `viterbi(x,Pi,A,B):` 

### B.1 Définition de la fonction viterbi

In [101]:
def viterbi(x,Pi,A,B):
    T = len(x)
    N = len(Pi)
    logPI = np.log(Pi)
    logA = np.log(A)
    logB = np.log(B)
    logdelta = np.zeros((N,T))
    psi = np.zeros((N,T))
    S = np.zeros(T) # les états à retourner
    # votre code
    # pass

    return S, logp # états + approximation de la vraimsemblance des obs par rapport au modèle

In [103]:
q, p = viterbi(Xd[0], Pi, A, B) # attention à bien donner la version discrétisée des observations

print(Xd[0],"\n",q, p)

[1. 9. 8. 8. 8. 8. 8. 9. 3. 4. 5. 6. 6. 6. 7. 7. 8. 9. 0. 0. 0. 1. 1.] 
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 4. 4. 4. 4. 4.] -38.09356554559258


/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


Check:
```
# décodage
[1. 9. 8. 8. 8. 8. 8. 9. 3. 4. 5. 6. 6. 6. 7. 7. 8. 9. 0. 0. 0. 1. 1.] 
 [0. 0. 0. 0. 0. 0. 0. 0. 1. 2. 2. 2. 2. 2. 3. 3. 3. 3. 4. 4. 4. 4. 4.] 
# log vraisemblance
-38.09356554559258
```

### B2. Utilisation de l'algorithme de Viterbi pour la classification

Nous aurions normalement du utiliser l'algorithme forward pour estimer la vraisemblance... Mais l'algorithme de Viterbi est plus versatile et il nous permettra de mettre en oeuvre des stratégies d'apprentissage dans les questions suivantes.

Nous allons donc utiliser la vraisemblance du chemin le plus probable (dans l'espace des états) comme une approximation de la vraisemblance de l'ensemble des chemins.

1. Calculer un modèle par lettre
1. Estimer la vraisemblance des lettres dans chaque modèle
1. Donner un score de classification

In [104]:
# calcul des 26 modèles
models = 

# affectation des signaux = calcul de vraisemblance[n_sig x n_modèles]
all_proba = 

/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


In [110]:
ypred = np.array(all_proba).argmax(1) # à modifier si votre all_proba est transposé

traduction = {ch:i for i,ch in enumerate(np.unique(Y))}
Ynum = np.array([traduction[y] for y in Y])

tx_bonne_classif = np.where(ypred == Ynum,1.,0).mean()
print("Taux de bonne classification" , tx_bonne_classif)

Taux de bonne classification 0.8507462686567164


Check : 85.07% de bonne classification

### B3. Séparation apprentissage / test

L'expérience précédente n'est pas très satisfaisante, l'évalution des performances n'étant pas fiable.
Nous allons donc séparer les données en un jeu d'apprentissage et un jeu de test.

In [124]:
# TOUT LE CODE EST FOURNI ICI

# séparation stratifiée par classe (non randomisée)
def separation_app_test(Y, pc_app):
    ind_app  = []
    ind_test = []
    for cl in np.unique(Y):
        index = np.where(Y == cl)[0]
        ind_app += index[:int(len(index)*pc_app)].tolist()
        ind_test += index[int(len(index)*pc_app):].tolist()
    return ind_app, ind_test

pc_app = 0.8 # pourcentage de points en apprentissage
ind_app, ind_test = separation_app_test(Y, pc_app)
# separation des données
Xd_app, Xd_test = Xd[ind_app], Xd[ind_test]
Y_app, Y_test   = Y[ind_app], Y[ind_test]
Yn_app, Yn_test = Ynum[ind_app], Ynum[ind_test]
Q_app, Q_test   = Q[ind_app], Q[ind_test]

In [126]:
# apprentissage des modèles sur les données d'apprentissage seulement

models = 

# calcul des affectations
all_proba_app = 
all_proba_test = 

# calcul des performances
ypred_app  = np.array(all_proba_app).argmax(1) # a verifier en fonction de vos amtrice all_proba
ypred_test = np.array(all_proba_test).argmax(1)
tx_bonne_classif_app = np.where(ypred_app == Yn_app,1.,0).mean()
tx_bonne_classif_test = np.where(ypred_test == Yn_test,1.,0).mean()
print("Taux de bonne classification" , tx_bonne_classif_app, tx_bonne_classif_test)

/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.


Taux de bonne classification 0.8509615384615384 0.7666666666666667


Check :

Taux de bonne classification 0.8509615384615384 
0.7666666666666667

### B4. Matrice de confusion

Afin de mieux comprendre les erreurs, tracer la matrice de confusion associée aux lettre


In [ ]:
def mat_conf(Y, Yhat):
    nCl = len(np.unique(Y)) # on espere que le vecteur des Y contient toutes les valeurs entre 0 et nCl-1
    conf = np.zeros((nCl,nCl))
    for y,yh in zip(Y,Yhat):
        conf[y, yh] += 1
    return conf

# appel à la fonction


### B5. OPT. Trouver les paramètres de discrétisation et de nombre d'états optimaux vis à vis des données de test

Cette procédrure s'appelle un grid-search: proposer des valeurs puis tester de manière exhaustive. 
Cette procédure est plutot simple à implémenter... mais longue en temps de calcul.

**NE PAS FAIRE EN TP**

**Note:** à force de faire des évaluations sur les données de test, on introduit un risque de sur-apprentissage de ces données.

## C. Baum-Welch simplifié

En utilisant la procédure de Baum-Welch simplifiée vue en TD et rappelée ci-dessous, proposer un code pour apprendre (ou plutot raffiner) les modèles correspondant aux 26 lettres de l'alphabet.

**Baum-Welch simplifié:**
1. Initialisation des états cachés arbitraire (eg méthode gauche-droite)
1. Tant que ''critère de convergence'' non atteint
    1. Apprentissage des modèles $\lambda_{lettre}=\{\Pi, A, B\}$
    1. Estimation des états cachés par Viterbi

Le critère de convergence sera la vraisemblance.
* A chaque itération $k$ et pour toutes les lettres $lettre$, calculer pour l'ensemble des séquences d'observation : 
$$\log\mathcal L^k = \sum_{lettre}\sum_i \log p(\mathbf x_i^{lettre} | \lambda_{lettre}^k)$$
* Lorsque la vraisemblance n'évolue plus (ie $\frac{\log\mathcal L^k - \log\mathcal L^{k+1}}{\log\mathcal L^k} < 1e-4$), sortir de la boucle de mise à jour.


* Donner l'implémentation de la méthode d'apprentissage
* Tracer la courbe de l'évolution de la vraisemblance au cours des itérations

### C1. Donner le code de la fonction baumwelch

In [133]:
# nIter : nombre d'iteration max (pour éviter les boucles infinies)
# Xd : les données d'observations discrétisées
# Yn : les étiquettes au format numérique
# models: une initialisation maline des modèles à apprendre
#     - RAPPEL: l'initialisation est critique !

def baumwelch(nIter, Xd, Yn, models):
    N = len(models[0][0])
    K = models[0][2].shape[1] # la seconde dimension de B
    print(N,K)
    # Baum Welch (simplifie)
    nCl = len(np.unique(Y))
    nIter = 10
    # A compléter
    
    return models

models = baumwelch(30, Xd_app, Yn_app, models)


5 10
0 -4448.105413831695
1 -3723.2800152952045


/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


2 -3718.5731479934116
3 -3713.054181192417
4 -3711.8252459541063
5 -3711.8252459541063
cvg in 5 iterations


### C2. Evaluation des performances

Les performances sont-elles meilleures après optimisation du modèle?

Faudrait-il refaire une procédure de grid-search basée sur l'algorithme Baum-Welch?

In [134]:
# calcul des affectations en apprentissage et en test
all_proba_app = 
all_proba_test = 

# calcul des perfromances
ypred_app  = np.array(all_proba_app).argmax(1)
ypred_test = np.array(all_proba_test).argmax(1)
tx_bonne_classif_app = np.where(ypred_app == Yn_app,1.,0).mean()
tx_bonne_classif_test = np.where(ypred_test == Yn_test,1.,0).mean()
print("Taux de bonne classification" , tx_bonne_classif_app, tx_bonne_classif_test)

/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: divide by zero encountered in log
  after removing the cwd from sys.path.
/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in log
  """
/Users/vguigue/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: RuntimeWarning: divide by zero encountered in log
  if sys.path[0] == '':


Taux de bonne classification 0.8653846153846154 0.7166666666666667


## D. Génération de lettres

De la même qu'un modèle de Markov, la chaine de Markov cachée est un modèle génératif: il est possible de générer des lettre.

Avec la version discrétisée des angles, ça ne va toujours pas donner de résultats formidables... Mais c'est possible